# Plotting gridded data

First up let's import PyGMT and some other stuff

In [ ]:
import pygmt
import pandas as pd
%matplotlib widget

We are going to play around with some model output Simon Barker and colleagues worked up in their [paper](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2018GC008152) on modeling ash dispersal from future eruptions of Taupō supervolcano.

All the outputs from their models are included in an online repository, as any good, modern paper should! I have taken the liberty of including one model output in the data directory and we will start by playing with that, but you should explore the data repository and get other model outputs if you want.

The output of the model is ash thickness, and is modeled dependent on a few things, including wind direction.

Let's start by reading the data. The data is given in ESRI-ASCII format, which is used in GIS programmes like Arc. We've provided a function in `helpers` to convert this into a Pandas dataframe:

In [ ]:
from helpers.asc_to_df import transform_asc_to_df

datafile = "data/DepositFile_ESRI_ASCII.txt"
df = transform_asc_to_df(datafile)
df.rename(columns={'variable':'thickness'}, inplace=True)

Now let's draw a map and some contours of ash thickness. For the contouring we use the aptly named `contour` module:

In [ ]:
fig=pygmt.Figure()
pygmt.config(MAP_FRAME_TYPE='plain', FORMAT_GEO_MAP='ddd.xx')
fig.coast(region=[173,179,-42,-36],
          shorelines=True,
          land='lightgreen',
          water='lightblue',
          projection='M10c',
          frame=['WSne','xa2f1','ya2f1'])

fig.contour(x=df['longitude'],
            y=df['latitude'],
            z=df['thickness'],
            levels=1, # draw a contour every 1mm thickness
            annotation=10, # annotate every 10mm
            pen='2p')

fig.show()

That must have been a strong Westerly. Poor Gisbourne...

There are really too many contours drawn there and it looks very messy. Let's tell PyGMT exactly which contours to plot. To do this we need to give the `levels` paramater a comma-seperated string of the contours we want. This isn't very pythonic, but hopefully this is improved in future versions.

In [ ]:
fig=pygmt.Figure()
pygmt.config(MAP_FRAME_TYPE='plain', FORMAT_GEO_MAP='ddd.xx')
fig.coast(region=[173,179,-42,-36],
          shorelines=True,
          land='lightgreen',
          water='lightblue',
          projection='M10c',
          frame=['WSne','xa2f1','ya2f1'])

fig.contour(x=df['longitude'],
            y=df['latitude'],
            z=df['thickness'],
            levels='1,5,10,15,20,25,30',
            annotation='1,5,10,15,20,25,30',
            pen='2p')

fig.show()

That's pretty easy and very handy for many earth science datasets. You can do something very similar with topography using the `grdcontour` module.

Now, go and download a different model run from Simon's [repository](https://www.sciencebase.gov/catalog/item/5cdd9ed7e4b029273746367a) and plot it up:

In [ ]:
### Your answer here

# Plotting Vectors

Another common observation we deal with in Earth Science is vectors. PyGMT can handle these really nicely.

For this exercise we're going to go down to Kaikōura and will begin by plotting an arbitrary vector with the `velo` module. We first have to out our data into a `dataframe` and then we plot it:

In [ ]:
fig=pygmt.Figure()
pygmt.config(MAP_FRAME_TYPE='plain', FORMAT_GEO_MAP='ddd.xx')
fig.coast(region=[172.5,174.5,-43,-41.5],
          shorelines=True,
          land='lightgreen',
          water='lightblue',
          projection='M10c',
          frame=['WSne','xa0.5f0.1','ya0.5f0.1'])

# Make a dataframe to store our data
df = pd.DataFrame(data={
        "x": [173.5],
        "y": [-42],
        "east_velocity": [3],
        "north_velocity": [3],
        })

fig.velo(data=df,
         pen='1p,black', # vector line
         color='black', # arrow head
         spec='e0.5/0.5', # this is telling it to plot a vector with a scale of 0.5 cm for each unit
         )

fig.show()

Ok great, but that vector doesn't mean anything. Let's do something more interesting.

Let's begin by importing some GNSS data using the function from our previous notebooks:

In [ ]:
import matplotlib.pyplot as plt
import requests  # This helps with web-requests
import datetime  # Python's representation of dates and times.


# The following block of code defines a function that we can use
# as often as we like to get GNSS data for a particular station
def get_gnss_for_station(
    station: str, 
    fits_url: str = "http://fits.geonet.org.nz/observation",) -> dict:
    """
    Get GNSS data from GeoNet for the station
    
    Parameters
    ----------
    station
        The name of the station you want to get data for
    fits_url
        URL of the FITS data service you want to query.
        
    Returns
    -------
    Dictionary with keys:
        time 
            list of timestamps of observations
        north
            list of offsets in mm in the north direction
        east
            list of offsets in mm in the east direction
        up          
            list of vertical offsets in mm
        north_error
            list of errors in mm for north
        east_error
            list of errors in mm for east
        up_error
            list of erros in mm for up
    
    """
    # Initialise an empty dictionary that we will append to
    out = dict(time=[],
               north=[],
               east=[],
               up=[],
               north_error=[],
               east_error=[],
               up_error=[])
    for channel in {"north", "east", "up"}:
        parameters = {"typeID": channel[0], "siteID": station}
        response = requests.get(fits_url, params=parameters)
        assert response.status_code == 200, "Bad request"
        payload = response.content.decode("utf-8").split("\n")
        # payload is a csv with header
        # This is a list-comprehension, a type of fast, one-line for loop
        payload = [p.split(',') for p in payload]
        # Check that this is what we expect
        assert payload[0][0] == 'date-time', "Unkown format"
        assert len(payload[0]) == 3, "Unknown format"
        times, displacements, errors = zip(*[
            (datetime.datetime.strptime(p[0], '%Y-%m-%dT%H:%M:%S.%fZ'),
             float(p[1]), float(p[2])) for p in payload[1:-1]])
        if len(out["time"]) == 0:
            out.update({"time": times})
        else:
            assert out["time"] == times, "Different time sampling for different components."
        out.update({channel: displacements, f"{channel}_error": errors})
    return out

Let's use that to download some data from a Kaikōura GNSS station, remember you can see the map of GNSS stations [here](https://www.geonet.org.nz/data/gnss/map):

In [ ]:
GNSS_data = pd.DataFrame(get_gnss_for_station('KAIK'))

Nice, now let's have a look at that GNSS data either side of the Kaikōura earthquake. Remember this earthquake happened at 11:02:56 on 2016-11-13 (UTC time):

In [ ]:
fig, ax = plt.subplots()

for component in ["north", "east", "up"]:
    ax.plot(GNSS_data["time"], GNSS_data[component], 
            label=component)

ax.set_xlim(pd.to_datetime('2016/11/06T11:02:56'), pd.to_datetime('2016/11/20T11:02:56')) # Calum is this the best way to be doing this?
ax.set_xlabel("Sample time (UTC)")
ax.set_ylabel("Offset in mm")
ax.set_title("GNSS offsets for KAIK")
ax.legend()
plt.show()

That's quite an offset! We could calculate a vector from that displacement

Let's first cut our dataset down to the two week period we're interested in. We can do this by defining a filter and using the `loc` option in `pandas`:

In [ ]:
date_filter = (GNSS_data['time'] >= '2016/11/06T11:02:56') & (GNSS_data['time'] <= '2016/11/20T11:02:56')

GNSS_data.loc[date_filter]

You can see that we've filtered our dataframe so that we now only have the data covering that two week period. Now to calculate the velocity over this time-window we simply need to calculate the difference between the north and east components at the beginning and end of this time period. Here's an example of how to do this for the North component:

In [ ]:
date_filter = (GNSS_data['time'] >= '2016/11/06T11:02:56') & (GNSS_data['time'] <= '2016/11/20T11:02:56')

N_first_value = GNSS_data.loc[date_filter]['north'].first_valid_index()
N_last_value = GNSS_data.loc[date_filter]['north'].last_valid_index()

N_velo = N_last_value - N_first_value

print(N_velo)

Now here's an exercise. Calculate both the North and East velocity for this GNSS station and plot it as a vector on the map. To give you a hand I've given you some code to download the GNSS locations from GeoNet and an example of how to get the Latitude for KAIK:

In [ ]:
# Downloading the GNSS locations from GeoNet's GitHub
url = (r'https://raw.githubusercontent.com/GeoNet/delta/main/network/marks.csv')
sites = pd.read_csv(url, delimiter=",")
sites.head()
sites = sites.reset_index()

site_lat = sites.loc[sites['Mark'] == 'KAIK']['Latitude']


### Your answer here





Great, now write a loop that downloads data and calculates vectors for the following nearby GNSS sites (MRBL, HANM, CMBL, WITH) and then plot them on a map.

You could also plot the Kaikōura [epicentre](https://www.geonet.org.nz/earthquake/2016p858000):

In [ ]:
### Your answer here

